In [ ]:
# Analysis of trends for CalOEHA report
## 1/4 deg OISST anoomalies data from 
## https://psl.noaa.gov/cgi-bin/db_search/DBSearch.pl?Dataset=NOAA+High-resolution+Blended+Analysis&Variable=Sea+Surface+Temperature+Anomalies&group=0&submit=Search
## Downloaded Jan 25, 2022

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
from scipy import stats
import warnings 
warnings.simplefilter('ignore') 

In [ ]:
ds = xr.open_mfdataset('./OISST*.nc')
ds

In [ ]:
# annual means
dsa=ds.resample(time='12MS').mean()
dsa

In [ ]:
# trends
lats=dsa.lat.values
lons=dsa.lon.values
x=np.arange(len(dsa.time.data))
trs = np.full((len(lats),len(lons)),np.nan)
trs2 = np.full((len(lats),len(lons)),np.nan)

for j, jns in enumerate(lats):
    print(str(jns)+'N')
    for i,ins in enumerate(lons):
        tmp = dsa.anom[:,j,i]
        if ~np.isnan(tmp[0]):
            y=tmp.data
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

            slope2 = slope*10 # change per decade
            trs[j,i]=slope2
            if p_value<=0.05:
                trs2[j,i]=1
                
dtr = xr.DataArray(trs, coords=[lats, lons], dims=['lat','lon'])
dtrs2 = xr.DataArray(trs2, coords=[lats, lons], dims=['lat','lon'])

In [ ]:
#  trends map
fig=plt.figure(figsize=(7,6),dpi=200)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m',linewidth=1,color='black',alpha=0.8,zorder=3)
ax.add_feature(cfeature.STATES, zorder=0)
ax.set_extent([232,244,31,43],crs=ccrs.PlateCarree())
ax.set_xticks([-128,-124,-120,-116], crs=ccrs.PlateCarree())
ax.set_yticks([32, 34, 36, 38, 40, 42], crs=ccrs.PlateCarree())
#ax.add_feature(cartopy.feature.LAND, zorder=2, edgecolor='black', color='lightgrey', alpha=0.8)

dtr.plot(alpha=0.9, cmap='bwr',zorder=1)

xs, ys = np.meshgrid(ds.lon, ds.lat)
im = plt.scatter(xs,ys,8,dtrs2, marker='x',alpha=0.7, cmap='gray') # p<0.05

ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.grid(True, zorder=0,alpha=0.5)
plt.title('')
ax.set_aspect(1.5)
plt.tight_layout()
plt.savefig('trends_1982-2021_p95.png')
plt.show()